In [1]:
# univariate multi-step vector-output 1d cnn example
from numpy import array
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import pandas as pd

from math import log
from math import exp
from scipy.stats import boxcox

In [2]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		# check if we are beyond the sequence
		if out_end_ix > len(sequence):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [3]:
# invert a boxcox transform for one value
def invert_boxcox(value, lam):
	# log case
	if lam == 0:
		return exp(value)
	# all other cases
	return exp(log(lam * value + 1) / lam)

In [4]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))

data = pd.read_csv("Timeseries_real_data_interview.csv")
t = data.iloc[0:11964] #11565
v = data.iloc[11964:19940]
ts = data.iloc[19940:20001]
# define input sequence
train_seq = t['Value'].values
val_seq = v['Value'].values
test_seq = ts['Value'].values

train_seq, tr_lamb = boxcox(train_seq)
val_seq, vl_lamb = boxcox(val_seq)
test_seq, ts_lamb = boxcox(test_seq)

train_seq = train_seq.reshape((len(train_seq), 1))
val_seq = val_seq.reshape((len(val_seq), 1))
test_seq = test_seq.reshape((len(test_seq), 1))

scaler1 = scaler.fit(train_seq)
train_seq = scaler1.transform(train_seq)

scaler2 = scaler.fit(val_seq)
val_seq = scaler2.transform(val_seq)

scaler3 = scaler.fit(test_seq)
test_seq = scaler3.transform(test_seq)

"""
tr, tr_lamb = boxcox(train_seq)
vl, vl_lamb = boxcox(val_seq)
ts, ts_lamb = boxcox(test_seq)
"""

print('Training Sequence : %d'%len(train_seq))
print('Validation Sequence : %d'%len(val_seq))
print('Testing Sequence : %d'%len(test_seq))

s1 = []
s2 = []
s3 = []

for i in range(len(train_seq)):
    s1.append(train_seq[i][0])
    
for i in range(len(val_seq)):
    s2.append(val_seq[i][0])
    
for i in range(len(test_seq)):
    s3.append(test_seq[i][0])
    
tr = s1
vl = s2
ts = s3

Training Sequence : 11964
Validation Sequence : 7976
Testing Sequence : 60


In [5]:
# choose a number of time steps
n_steps_in, n_steps_out = 30, 30
# split into samples
X_train, y_train = split_sequence(tr, n_steps_in, n_steps_out)
X_val, y_val = split_sequence(vl, n_steps_in, n_steps_out)
X_test, y_test = split_sequence(ts, n_steps_in, n_steps_out)
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1

In [6]:
# define model
import keras.backend as kb

def r2_score(y_true, y_pred):
    ss_res = kb.sum(kb.square(y_true - y_pred))
    ss_tot = kb.sum(kb.square(y_true - kb.mean(y_true)))
    return (1 - ss_res/(ss_tot + kb.epsilon()))

model = Sequential()
model.add(Dense(140, activation='elu', input_dim=n_steps_in))
model.add(Dropout(0.2))
model.add(Dense(n_steps_out))


adam = keras.optimizers.Adam(lr=0.001)
model.compile(optimizer=adam, loss='mse', metrics=[r2_score,'mae', keras.metrics.RootMeanSquaredError()])

In [ ]:
# fit model
history = model.fit(X_train, y_train, epochs=100, verbose=1, validation_data=(X_val, y_val), batch_size=64)

Epoch 1/100
187/187 [==============================] - 2s 11ms/step - loss: 0.0178 - r2_score: 0.0404 - mae: 0.0946 - root_mean_squared_error: 0.1336 - val_loss: 0.0039 - val_r2_score: 0.7441 - val_mae: 0.0448 - val_root_mean_squared_error: 0.0622
Epoch 2/100
187/187 [==============================] - 2s 8ms/step - loss: 0.0059 - r2_score: 0.6809 - mae: 0.0593 - root_mean_squared_error: 0.0770 - val_loss: 0.0033 - val_r2_score: 0.7862 - val_mae: 0.0428 - val_root_mean_squared_error: 0.0571
Epoch 3/100
187/187 [==============================] - 2s 9ms/step - loss: 0.0047 - r2_score: 0.7457 - mae: 0.0523 - root_mean_squared_error: 0.0687 - val_loss: 0.0030 - val_r2_score: 0.8030 - val_mae: 0.0393 - val_root_mean_squared_error: 0.0548
Epoch 4/100
187/187 [==============================] - 1s 8ms/step - loss: 0.0041 - r2_score: 0.7804 - mae: 0.0482 - root_mean_squared_error: 0.0639 - val_loss: 0.0028 - val_r2_score: 0.8143 - val_mae: 0.0376 - val_root_mean_squared_error: 0.0532
Epoch 5/100

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('MSE - MLP Model')
plt.ylabel('MSE')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.savefig('saved/mlp/loss.png')
plt.show()

In [ ]:
plt.plot(history.history['r2_score'])
plt.plot(history.history['val_r2_score'])
plt.title('R2 Score - MLP Model')
plt.ylabel('R2 Score')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.savefig('saved/mlp/r2_score.png')
plt.show()

In [ ]:
plt.plot(history.history['mae'])
plt.plot(history.history['val_mae'])
plt.title('MAE - MLP Model')
plt.ylabel('MAE')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.savefig('saved/mlp/mae.png')
plt.show()

In [ ]:
plt.plot(history.history['root_mean_squared_error'])
plt.plot(history.history['val_root_mean_squared_error'])
plt.title('RMSE - MLP Model')
plt.ylabel('RMSE')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.savefig('saved/mlp/rmse.png')
plt.show()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='saved/mlp/model.png', show_shapes=True, show_layer_names=True)

In [ ]:
print(model.summary())

In [ ]:
hist_df = pd.DataFrame(history.history) 
hist_csv_file = 'saved/mlp/history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [ ]:
prediction = model.predict(X_test, verbose=0)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(prediction, y_test)
r2 = r2_score(prediction[0], y_test[0])
print('Predicted MSE : %s'%mse)
print('Predicted R2 Score : %s'%r2)

In [ ]:
y_test[0]

In [ ]:
prediction[0]

In [ ]:
y_test_invert = scaler.inverse_transform(y_test)
y_test_invert
y_test_invert_final = []
# Inverse Box-Cox Transform
for i in range(len(y_test_invert[0])):
    y_test_invert_final.append(invert_boxcox(y_test_invert[0][i], ts_lamb))
x = np.arange(30)
plt.bar(x, y_test_invert_final)

In [ ]:
y_pred_invert = scaler.inverse_transform(prediction)

y_pred_invert_final = []
# Inverse Box-Cox Transform
for i in range(len(y_pred_invert[0])):
    y_pred_invert_final.append(invert_boxcox(y_pred_invert[0][i], ts_lamb))
x = np.arange(30)
plt.bar(x, y_pred_invert_final)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_pred_invert_final, y_test_invert_final)
r2 = r2_score(y_pred_invert_final, y_test_invert_final)
print('Predicted MSE : %s'%mse)
print('Predicted R2 Score : %s'%r2)

In [ ]:
y_pred_invert

In [ ]:
y_test_invert

In [ ]:
plt.plot(y_test_invert[0])
plt.plot(y_pred_invert[0])
plt.title('CNN - 30 Day Forecast')
plt.ylabel('Rainfall')
plt.xlabel('Days')
plt.legend(['Actual', 'Predicted'], loc='upper left')
plt.savefig('saved/mlp/test.png')
plt.show()